A refaire

In [ ]:
import numpy as np
import pandas as pd



from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, GlobalAveragePooling2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam,Adagrad

from keras.applications.vgg16 import VGG16
from keras.applications import VGG19
from keras.applications import EfficientNetB0




from tensorflow.keras.applications.vgg16 import preprocess_input
import tensorflow as tf
from RktnChallenge.RktnModel import ModelTrainer



#for original,duplicate,_,_ in duplicates:    
#    trainer.data = trainer.data.drop(trainer#.data[trainer.data['productid'] == int(original)].index)
#    trainer.data = trainer.data.drop(trainer.data[trainer.data['productid'] == int(duplicate)].index)
    
import os     
from sklearn.preprocessing import MinMaxScaler
#get file length
import gzip
import io
from PIL import Image


2023-09-08 16:57:03.668010: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-08 16:57:03.701925: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-08 16:57:04.208231: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
PATH = "images/crop100/image_train/"
img_size = 100
depth = 3

preprocessing_function = tf.keras.applications.vgg16.preprocess_input
trainer = ModelTrainer("data.csv",PATH)
trainer.loadData(PATH=PATH,imgsize=100,batchsize=32,aumgentImages=False,
                preprocessing_function=preprocessing_function,customized_data=True,data=df_copy,)

print(len(trainer.dataLoader.data_train))

In [ ]:
net = None
def build_model():
    input = tf.keras.Input(shape=(img_size,img_size,3))
    net = VGG19(weights='imagenet', include_top=False,input_tensor = input) 



    for layer in net.layers[:-2]:
        layer.trainable = False
        
    gap = tf.keras.layers.GlobalAveragePooling2D()(net.output)
    norm2 = tf.keras.layers.BatchNormalization()(gap)

    dense1 = tf.keras.layers.Dense(512, activation='relu')(norm2)
    dropout1 = tf.keras.layers.Dropout(0.3)(dense1)

    dense2 = tf.keras.layers.Dense(124, activation='relu')(dropout1)
    dropout2 = tf.keras.layers.Dropout(0.3)(dense2)

    output = tf.keras.layers.Dense(num_categories, activation='softmax')(dropout2)




    base_model = tf.keras.Model(net.input, outputs = output)
    base_model.compile(loss=['categorical_crossentropy'], optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return base_model


model = build_model()

    

In [ ]:
trainer.train(model=model, epochs = 10)